In [4]:
import tifffile as tif
from collections import OrderedDict
import nrrd
import os
import matplotlib.pyplot as plt
import cv2
import re
import numpy as np
from pprint import pprint

from octvision3d.utils import get_filenames


In [5]:
dir_path = "/data/dkermany_data/3D-OCT/first-batch-labeled"

In [18]:
def create_dataset_dirs(path):
    for i in ["imagesTr", "imagesTs", "labelsTr", "labelsTs"]:
        if not os.path.exists(os.path.join(path, i)):
            os.makedirs(os.path.join(path, i), exist_ok=True)

SyntaxError: invalid syntax (2988276572.py, line 2)

In [ ]:
create_dataset_dirs(dir_path)

In [20]:
tif_paths = get_filenames(dir_path, ext="tif")
label_paths = get_filenames(dir_path, ext="seg.nrrd")
for tif_path, label_path in zip(tif_paths, label_paths):
    assert tif_path.split(".")[0] == label_path.split(".")[0]
    # TODO move tif file to imagesTr
    # TODO convert seg.nrrd to one-channel tif and move to labelsTr
    # TODO make json file to specify tif spacing